# Sprint14
## ディープラーニングフレームワーク

### ライブラリインポート

In [26]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder 
import matplotlib.pyplot as plt
%matplotlib inline
from keras.datasets import mnist
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.preprocessing import OneHotEncoder

## 【問題1】公式Exampleを分担して実行

TensorFLowの公式Exampleのresearch（定番のモデルから最新のモデルまで多様なコードが公開されている）ディレクトリから
"adversarial_crypto"を選択した。下記参考Git,文献である。

[adversarial_crypto](https://github.com/tensorflow/models/tree/master/research/adversarial_crypto)

[Learning to Protect Communications with Adversarial Neural Cryptography](https://arxiv.org/abs/1610.06918)

[論文PDF](https://arxiv.org/pdf/1610.06918.pdf)

### 概要

adversarial_crypto: 通信の自動暗号化

通信保護とデータ・セキュリティに応用できるモデル
特定の暗号アルゴリズムを規定しておらず、EndToEndの反復トレーニングにより、ニューラルネットワークがどのように暗号化と復号化を行うかを学習を実証

ニューラルネットワークが秘密鍵を使用して他のニューラルネットワークから情報を保護することを学習できるかどうかを確認します。 (公開鍵を使って暗号化すると秘密鍵でのみ復号できます)
具体的には、マルチエージェントシステムで機密性のプロパティを確保することに焦点を当て、それらのプロパティを敵の観点から指定します。 したがって、システムはアリスとボブという名前のニューラルネットワークで構成され、イブという名前の3番目のニューラルネットワークがアリスとボブ間の通信を盗聴することから学習するものを制限することを目指しています。
これらのニューラルネットワークに特定の暗号アルゴリズムを処方することはありません。 
代わりに、エンドツーエンドで敵対的にトレーニングします。 
ニューラルネットワークは、暗号化と復号化の形式を実行する方法、および機密性の目標を満たすためにこれらの操作を選択的に適用する方法を学習できることを示します。

**ニューラルネットワークと暗号化及び機密性の確保**
（2016年10月２４日Google公表）

**エンドツーエンドにおける三人のネットワーク構成**
- アリス：メッセージ送信者。ボブに暗号化したメッセージを送りたい。
- ボブ：メッセージ受信者。アリスから受け取った暗号文を復号したい。
- イヴ：受動的な攻撃者（メッセージの改ざんはしない）。アリスからボブへの通信を傍受し、暗号文を復号したい。

**通信手順**
- アリスは平文Pを暗号文Cに暗号化しボブに送る。
- ボブは暗号文Cを正確に複合し、複合文PBobを得たい。
- イブも暗号文Cを傍受し複合文PEveを得たい。

**登場人物をニューラルネットワークとして考える**
ここで、アリス、ボブ、イブの各登場人物をニューラルネットワークAlice,Bob,Eveとする。
- Aliceは平文Pとvを入力として受け取り暗号文Cを出力する
- Bobは暗号文Cと秘密鍵Kを入力として受け取り複合文PBobを出力する
- Eveは暗号文Cを入力として受け取り複合文PEveを出力する

**point**
暗号化の際に、特定の暗号アルゴリズムを指定して用いている訳ではない点。 
既知のアルゴリズムを用いていないため、攻撃者もニューラルネットワークを構築して複合化を試みる。 
また、攻撃者のニューラルネットワークEveは第三者的立場としての最善のモデルを使用することを仮定する。
AliceとBobの間には、Eveには見えないワンタイムパッドを共有。このワンタイムパッドでは、送信ごとに浮動小数点の暗号キーが生成される。AliceとBob双方には、このワンタイムパッドを使って暗号化するよう指示したが、具体的な方法については指示をしていない。共有のワンタイムパッドを使ったXOR(排他的理論和)暗号化が一般的だが、そういった機能の実装については指示しなかった。


**学習**
- 全結合層を最初の層にもつ
- 全結合層の後に畳み込み層のシーケンスが続き、最終的に平文または暗号文に適した サイズの出力を生成する

BobとEveには、この暗号を解読するよう指示、Bobは共有のワンタイムパッド（鍵）にアクセスできるが、Aliceがどういった暗号化手段を使っているのか分からないため、学習していく必要がある。
16bitのプレーンテキストを送信する際に、4,096個の例を用いて機械学習させていったところ、Bobは7,000回を超えた辺りからbitの誤りが急激に低下し、15,000回以降、0に近い値で推移したのに対し、Eveは最良時でも約5bit、平均して7～8bitの誤りがあったという。（半角文字１byte=8bit）

つまり、Bobは共有のワンタイムパッドから暗号化アルゴリズムを見出し、Aliceから送られた暗号をほぼ解読できたが、ワンタイムパッドが共有されていないEveでは内容を推測することしかできず、解読できなかったわけだ。
Googleは、こうした研究結果は、ニューラルネットワークは自身の通信を暗号化するだけでなく、外部攻撃からの保護にも活用できるとしている。

現代のRSA暗号やSSL暗号は、コンピュータの速度の制限によって強度を担保している暗号システムであるため、計算速度が猛烈に早い量子コンピュータが本格的に実用化した場合、秒で解読できる。
ニューラル暗号は、コンピュータの計算速度に依存せず、復号化キーなければわからない。 　

### コードの実行

In [3]:
# Copyright 2016 The TensorFlow Authors All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

"""Adversarial training to learn trivial encryption functions,
from the paper "Learning to Protect Communications with
Adversarial Neural Cryptography", Abadi & Andersen, 2016.
https://arxiv.org/abs/1610.06918
This program creates and trains three neural networks,
termed Alice, Bob, and Eve.  Alice takes inputs
in_m (message), in_k (key) and outputs 'ciphertext'.
Bob takes inputs in_k, ciphertext and tries to reconstruct
the message.
Eve is an adversarial network that takes input ciphertext
and also tries to reconstruct the message.
The main function attempts to train these networks and then
evaluates them, all on random plaintext and key values.
"""

# TensorFlow Python 3 compatibility
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import signal
import sys
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

# 定義用のflagを作成,tf.app.flags.FLAGSを使うと、TensorFlowのPythonファイルを実行する際にパラメタを付与できる
flags = tf.app.flags 

# floatの定義
flags.DEFINE_float('learning_rate', 0.0008, 'Constant learning rate')
# intの定義
flags.DEFINE_integer('batch_size', 4096, 'Batch size')

# 参照用のflagを作成
FLAGS = flags.FLAGS

# Input and output configuration.
TEXT_SIZE = 16
KEY_SIZE = 16

# Training parameters.
ITERS_PER_ACTOR = 1
EVE_MULTIPLIER = 2  # Train Eve 2x for every step of Alice/Bob

# Train until either max loops or Alice/Bob "good enough":
# Alice/Bobが十分になるまで学習を繰り返す
MAX_TRAINING_LOOPS = 850000
BOB_LOSS_THRESH = 0.02  # Exit when Bob loss < 0.02 and Eve > 7.7 bits
EVE_LOSS_THRESH = 7.7

# Logging and evaluation.
PRINT_EVERY = 200  # In training, log every 200 steps.　トレーニングでは２００ステップごとに記録
EVE_EXTRA_ROUNDS = 2000  # At end, train eve a bit more.　最後に、イブをもう少し訓練
RETRAIN_EVE_ITERS = 10000  # Retrain eve up to ITERS*LOOPS times.　ITERS * LOOPS回までイブを再トレーニングし
RETRAIN_EVE_LOOPS = 25  # With an evaluation each loop　ループごとに評価
NUMBER_OF_EVE_RESETS = 5  # And do this up to 5 times with a fresh eve.　洗練されたEveで５回回す
# Use EVAL_BATCHES samples each time we check accuracy.　EVAL_BATCHESを使いaccuracy確認
EVAL_BATCHES = 1


def batch_of_random_bools(batch_size, n):
  """Return a batch of random "boolean" numbers.
  Args:
    batch_size:  Batch size dimension of returned tensor.
    n:  number of entries per batch.
  Returns:
    A [batch_size, n] tensor of "boolean" numbers, where each number is
    preresented as -1 or 1.
  """

  as_int = tf.random.uniform([batch_size, n], minval=0, maxval=2, dtype=tf.int32)
    expanded_range = (as_int * 2) - 1
    
    return tf.cast(expanded_range, tf.float32) #tf.cast()でint型→float型へ変換。


class AdversarialCrypto(object):
  """Primary model implementation class for Adversarial Neural Crypto.
  This class contains the code for the model itself,
  and when created, plumbs the pathways from Alice to Bob and
  Eve, creates the optimizers and loss functions, etc.
  Attributes:
    eve_loss:  Eve's loss function.
    bob_loss:  Bob's loss function.  Different units from eve_loss.
    eve_optimizer:  A tf op that runs Eve's optimizer.
    bob_optimizer:  A tf op that runs Bob's optimizer.
    bob_reconstruction_loss:  Bob's message reconstruction loss,
      which is comparable to eve_loss.
    reset_eve_vars:  Execute this op to completely reset Eve.
    
    モデルの本体コードが含まれるクラス、実行されるとアリスからボブとイブへ経路が組込まれ、ロスやオプティマイザーが生成される。
  """

  def get_message_and_key(self):
    """Generate random pseudo-boolean key and message values."""
    #キーとメッセージ値生成

    batch_size = tf.compat.v1.placeholder_with_default(FLAGS.batch_size, shape=[])

    in_m = batch_of_random_bools(batch_size, TEXT_SIZE)
    in_k = batch_of_random_bools(batch_size, KEY_SIZE)
    return in_m, in_k

  def model(self, collection, message, key=None):
    """The model for Alice, Bob, and Eve.  If key=None, the first fully connected layer
    takes only the message as inputs.  Otherwise, it uses both the key
    and the message.
    Args:
      collection:  The graph keys collection to add new vars to.
      message:  The input message to process.
      key:  The input key (if any) to use.
    """

    if key is not None: #キーがある場合
        combined_message = tf.concat(axis=1, values=[message, key])
        #メッセージとキーを複合
    else:#キーがNoneの場合
        combined_message = message
        #最初に完全結合レイヤーはメッセージのみを入力として受け取る。 

    # Ensure that all variables created are in the specified collection.
    #作成した変数が指定されたコレクションにあることを確認。
    with tf.contrib.framework.arg_scope(
        [tf.contrib.layers.fully_connected, tf.contrib.layers.conv2d],
        variables_collections=[collection]):

        fc = tf.contrib.layers.fully_connected(
          combined_message,
          TEXT_SIZE + KEY_SIZE,
          biases_initializer=tf.constant_initializer(0.0),
          activation_fn=None)

        # Perform a sequence of 1D convolutions (by expanding the message out to 2D
        # and then squeezing it back down).
        fc = tf.expand_dims(fc, 2) # 2D
        fc = tf.expand_dims(fc, 3) # 3D -- conv2d needs a depth
        # 2,1 -> 1,2
        conv = tf.contrib.layers.conv2d(
          fc, 2, 2, 2, 'SAME', activation_fn=tf.nn.sigmoid)
        # 1,2 -> 1, 2
        conv = tf.contrib.layers.conv2d(
          conv, 2, 1, 1, 'SAME', activation_fn=tf.nn.sigmoid)
        # 1,2 -> 1, 1
        conv = tf.contrib.layers.conv2d(
          conv, 1, 1, 1, 'SAME', activation_fn=tf.nn.tanh)
        conv = tf.squeeze(conv, 3)
        conv = tf.squeeze(conv, 2)
        return conv

    def __init__(self):
        in_m, in_k = self.get_message_and_key()
        encrypted = self.model('alice', in_m, in_k)
        decrypted = self.model('bob', encrypted, in_k)
        eve_out = self.model('eve', encrypted, None)

        self.reset_eve_vars = tf.group(
            *[w.initializer for w in tf.compat.v1.get_collection('eve')])

        optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=FLAGS.learning_rate)

        # Eve's goal is to decrypt the entire message:
        eve_bits_wrong = tf.reduce_sum(
            tf.abs((eve_out + 1.0) / 2.0 - (in_m + 1.0) / 2.0), [1])
        self.eve_loss = tf.reduce_sum(eve_bits_wrong)
        self.eve_optimizer = optimizer.minimize(
            self.eve_loss, var_list=tf.compat.v1.get_collection('eve'))

        # Alice and Bob want to be accurate...
        self.bob_bits_wrong = tf.reduce_sum(
            tf.abs((decrypted + 1.0) / 2.0 - (in_m + 1.0) / 2.0), [1])
        # ... and to not let Eve do better than guessing.
        self.bob_reconstruction_loss = tf.reduce_sum(self.bob_bits_wrong)
        bob_eve_error_deviation = tf.abs(float(TEXT_SIZE) / 2.0 - eve_bits_wrong)
        # 7-9 bits wrong is OK too, so we squish the error function a bit.
        # Without doing this, we often tend to hang out at 0.25 / 7.5 error,
        # and it seems bad to have continued, high communication error.
        bob_eve_loss = tf.reduce_sum(
            tf.square(bob_eve_error_deviation) / (TEXT_SIZE / 2)**2)

        # Rescale the losses to [0, 1] per example and combine.
        self.bob_loss = (self.bob_reconstruction_loss / TEXT_SIZE + bob_eve_loss)

        self.bob_optimizer = optimizer.minimize(
            self.bob_loss,
            var_list=(tf.compat.v1.get_collection('alice') + tf.compat.v1.get_collection('bob')))


def doeval(s, ac, n, itercount):
  """Evaluate the current network on n batches of random examples.
  Args:
    s:  The current TensorFlow session
    ac: an instance of the AdversarialCrypto class
    n:  The number of iterations to run.
    itercount: Iteration count label for logging.
  Returns:
    Bob and Eve's loss, as a percent of bits incorrect.
  """

  bob_loss_accum = 0
  eve_loss_accum = 0
  for _ in xrange(n):
    bl, el = s.run([ac.bob_reconstruction_loss, ac.eve_loss])
    bob_loss_accum += bl
    eve_loss_accum += el
  bob_loss_percent = bob_loss_accum / (n * FLAGS.batch_size)
  eve_loss_percent = eve_loss_accum / (n * FLAGS.batch_size)
  print('%10d\t%20.2f\t%20.2f'%(itercount, bob_loss_percent, eve_loss_percent))
  sys.stdout.flush()
  return bob_loss_percent, eve_loss_percent


def train_until_thresh(s, ac):
    for j in xrange(MAX_TRAINING_LOOPS):
        for _ in xrange(ITERS_PER_ACTOR):
            s.run(ac.bob_optimizer)
        for _ in xrange(ITERS_PER_ACTOR * EVE_MULTIPLIER):
            s.run(ac.eve_optimizer)
        if j % PRINT_EVERY == 0:
            bob_avg_loss, eve_avg_loss = doeval(s, ac, EVAL_BATCHES, j)
            if (bob_avg_loss < BOB_LOSS_THRESH and eve_avg_loss > EVE_LOSS_THRESH):
                print('Target losses achieved.')
        return True
    return False


def train_and_evaluate():
  """Run the full training and evaluation loop."""

    ac = AdversarialCrypto()
    init = tf.compat.v1.global_variables_initializer()

    with tf.compat.v1.Session() as s:
        s.run(init)
        print('# Batch size: ', FLAGS.batch_size)
        print('# %10s\t%20s\t%20s'%("Iter","Bob_Recon_Error","Eve_Recon_Error"))

    if train_until_thresh(s, ac):
        for _ in xrange(EVE_EXTRA_ROUNDS):
            s.run(ac.eve_optimizer)
        print('Loss after eve extra training:')
            doeval(s, ac, EVAL_BATCHES * 2, 0)
        for _ in xrange(NUMBER_OF_EVE_RESETS):
            print('Resetting Eve')
            s.run(ac.reset_eve_vars)
            eve_counter = 0
            for _ in xrange(RETRAIN_EVE_LOOPS):
                for _ in xrange(RETRAIN_EVE_ITERS):
                    eve_counter += 1
                    s.run(ac.eve_optimizer)
                doeval(s, ac, EVAL_BATCHES, eve_counter)
            doeval(s, ac, EVAL_BATCHES, eve_counter)


def main(unused_argv):
  # Exit more quietly with Ctrl-C.
  signal.signal(signal.SIGINT, signal.SIG_DFL)
    train_and_evaluate()


if __name__ == '__main__':
    tf.compat.v1.app.run()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W1210 08:00:49.834367 4552873408 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



# Batch size:  4096
#       Iter	     Bob_Recon_Error	     Eve_Recon_Error
         0	                8.01	                7.98
       200	                7.79	                7.38
       400	                6.07	                6.22
       600	                4.29	                5.89
       800	                2.26	                5.84
      1000	                1.44	                6.07
      1200	                1.00	                6.27
      1400	                0.73	                6.46
      1600	                0.57	                6.52
      1800	                0.46	                6.72
      2000	                0.38	                6.77
      2200	                0.32	                6.86
      2400	                0.27	                6.97
      2600	                0.23	                7.03
      2800	                0.19	                7.07
      3000	                0.17	                7.11
      3200	                0.15	                7.15
      3400	             

SystemExit: 

/Users/wakaichiaki/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3304: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


上記の結果から


現代のRSA暗号やSSL暗号は、コンピュータの速度の制限によって強度を担保している暗号システムであるため、計算速度が猛烈に早い量子コンピュータが本格的に実用化した場合、一瞬で解読される。
ニューラル暗号は、コンピュータの計算速度に依存せず、復号化キーなければわからない。 　

### 論文（英文）

### 論文直訳

## 【問題2】Iris（2値分類）をKerasで学習

In [3]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""

# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)


K.clear_session()

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(50, input_shape=(4,), activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(100, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.compile(loss="binary_crossentropy",
              optimizer=tf.train.AdamOptimizer(learning_rate=0.01),
              metrics=["accuracy"])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                250       
_________________________________________________________________
dense_1 (Dense)              (None, 100)               5100      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 5,451
Trainable params: 5,451
Non-trainable params: 0
_________________________________________________________________


In [5]:
history=model.fit(X_train,y_train,batch_size=10,epochs=10,verbose=1)

Epoch 1/10
64/64 [==============================] - 0s 2ms/sample - loss: 0.9648 - acc: 0.4844
Epoch 2/10
64/64 [==============================] - 0s 295us/sample - loss: 0.7601 - acc: 0.5469
Epoch 3/10
64/64 [==============================] - 0s 320us/sample - loss: 0.6639 - acc: 0.5156
Epoch 4/10
64/64 [==============================] - 0s 245us/sample - loss: 0.6510 - acc: 0.5625
Epoch 5/10
64/64 [==============================] - 0s 207us/sample - loss: 0.5684 - acc: 0.7656
Epoch 6/10
64/64 [==============================] - 0s 327us/sample - loss: 0.5527 - acc: 0.7656
Epoch 7/10
64/64 [==============================] - 0s 261us/sample - loss: 0.4938 - acc: 0.7812
Epoch 8/10
64/64 [==============================] - 0s 248us/sample - loss: 0.4683 - acc: 0.7812
Epoch 9/10
64/64 [==============================] - 0s 285us/sample - loss: 0.4842 - acc: 0.7188
Epoch 10/10
64/64 [==============================] - 0s 278us/sample - loss: 0.4524 - acc: 0.7812


In [6]:
y_pred_proba = model.predict(X_test)
y_pred = np.where(y_pred_proba >0.5, 1, 0)
print(accuracy_score(y_test,y_pred))

0.8


## 【問題3】Iris（多値分類）をKerasで学習

In [6]:
# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-setosa'] = 0
y[y=='Iris-versicolor'] = 1
y[y=='Iris-virginica'] = 2
y = y.astype(np.int)[:, np.newaxis]

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

#OneHot
enc = OneHotEncoder(handle_unknown="ignore", sparse=False)
y_train = enc.fit_transform(y_train)
y_test = enc.fit_transform(y_test)
y_val = enc.fit_transform(y_val)

In [7]:
K.clear_session()

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(50, input_shape=(4,), activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(100, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(3, activation=tf.nn.softmax))
model.compile(loss="categorical_crossentropy",
              optimizer=tf.train.AdamOptimizer(learning_rate=0.01),
              metrics=["accuracy"])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


W1210 19:31:19.031019 4552873408 deprecation.py:506] From /Users/wakaichiaki/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                250       
_________________________________________________________________
dense_1 (Dense)              (None, 100)               5100      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 303       
Total params: 5,653
Trainable params: 5,653
Non-trainable params: 0
_________________________________________________________________


In [9]:
history=model.fit(X_train,y_train,batch_size=10,epochs=10,verbose=1)

Epoch 1/10
96/96 [==============================] - 0s 1ms/sample - loss: 0.6381 - acc: 0.6875
Epoch 2/10
96/96 [==============================] - 0s 216us/sample - loss: 0.3946 - acc: 0.7917
Epoch 3/10
96/96 [==============================] - 0s 292us/sample - loss: 0.2850 - acc: 0.8438
Epoch 4/10
96/96 [==============================] - 0s 412us/sample - loss: 0.2227 - acc: 0.8958
Epoch 5/10
96/96 [==============================] - 0s 163us/sample - loss: 0.1172 - acc: 0.9583
Epoch 6/10
96/96 [==============================] - 0s 269us/sample - loss: 0.1133 - acc: 0.9479
Epoch 7/10
96/96 [==============================] - 0s 178us/sample - loss: 0.1502 - acc: 0.9375
Epoch 8/10
96/96 [==============================] - 0s 257us/sample - loss: 0.1101 - acc: 0.9479
Epoch 9/10
96/96 [==============================] - 0s 260us/sample - loss: 0.0637 - acc: 0.9792
Epoch 10/10
96/96 [==============================] - 0s 287us/sample - loss: 0.0676 - acc: 0.9792


In [19]:
y_pred_proba=model.predict(X_test)
y_pred = np.argmax(y_pred_proba, axis=1)
print("accuracy",accuracy_score(np.argmax(y_test,axis=1),y_pred))

accuracy 1.0


## 【問題4】House PricesをKerasで学習

In [20]:
df = pd.read_csv("train.csv")
X = df.loc[:, ["GrLivArea", "YearBuilt"]].values
y = df["SalePrice"].values
y = np.log(y)
X = np.log(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [29]:
K.clear_session()

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(200, input_dim=2, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(100, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(1))
model.compile(loss="mean_squared_error",optimizer = tf.train.AdamOptimizer(learning_rate=0.01),
             metrics=["mse"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 200)               600       
_________________________________________________________________
dense_1 (Dense)              (None, 100)               20100     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 20,801
Trainable params: 20,801
Non-trainable params: 0
_________________________________________________________________


In [30]:
history=model.fit(X_train,y_train,batch_size=1,epochs=100,verbose=0)

In [28]:
y_pred=model.predict(X_test)
print(np.sqrt(mean_squared_error(y_test, y_pred)))

0.4321411340072334


## 【問題5】MNISTをKerasで学習

In [ ]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 110s 2ms/step - loss: 0.2675 - accuracy: 0.9180 - val_loss: 0.0569 - val_accuracy: 0.9815
Epoch 2/12
60000/60000 [==============================] - 122s 2ms/step - loss: 0.0851 - accuracy: 0.9749 - val_loss: 0.0418 - val_accuracy: 0.9851
Epoch 3/12
60000/60000 [==============================] - 132s 2ms/step - loss: 0.0646 - accuracy: 0.9805 - val_loss: 0.0336 - val_accuracy: 0.9880
Epoch 4/12
58368/60000 [============================>.] - ETA: 3s - loss: 0.0510 - accuracy: 0.9851